During the course of doing data analysis and modeling, a significant amount of time
is spent on data preparation: loading, cleaning, transforming, and rearranging. Such
tasks are often reported to take up 80% or more of an analyst’s time. Sometimes the
way that data is stored in files or databases is not in the right format for a particular
task. Many researchers choose to do ad hoc processing of data from one form to
another using a general-purpose programming language, like Python, Perl, R, or Java,
or Unix text-processing tools like sed or awk. Fortunately, pandas, along with the
built-in Python language features, provides you with a high-level, flexible, and fast set
of tools to enable you to manipulate data into the right form

## 7.1 Handling Missing Data

Missing data occurs commonly in many data analysis applications. One of the goals
of pandas is to make working with missing data as painless as possible. For example,
all of the descriptive statistics on pandas objects exclude missing data by default.
    
      
The way that missing data is represented in pandas objects is somewhat imperfect,
but it is functional for a lot of users. For numeric data, pandas uses the floating-point
value NaN (Not a Number) to represent missing data. We call this a sentinel value that
can be easily detected:

In [1]:
import pandas as pd
import numpy as np
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])

In [2]:
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [3]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In pandas, we’ve adopted a convention used in the R programming language by referring to missing data as NA, which stands for not available. In statistics applications,
NA data may either be data that does not exist or that exists but was not observed
(through problems with data collection, for example). When cleaning up data for
analysis, it is often important to do analysis on the missing data itself to identify data
collection problems or potential biases in the data caused by missing data.

The built-in Python None value is also treated as NA in object arrays:

In [4]:
string_data[0] = None

In [5]:
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

There is work ongoing in the pandas project to improve the internal details of how
missing data is handled, but the user API functions, like pandas.isnull, abstract
away many of the annoying details.

Table 7-1. NA handling methods
--------------------------------------------
Argument --> Description
   
dropna --> Filter axis labels based on whether values for each label have missing data, with varying thresholds for how
much missing data to tolerate.   
   
fillna --> Fill in missing data with some value or using an interpolation method such as 'ffill' or 'bfill'.
    
      
isnull --> Return boolean values indicating which values are missing/NA.
    
     
notnull --> Negation of isnull.

## Filtering Out Missing Data

There are a few ways to filter out missing data. While you always have the option to
do it by hand using pandas.isnull and boolean indexing, the dropna can be helpful.
On a Series, it returns the Series with only the non-null data and index values:

In [6]:
from numpy import nan as NA

In [7]:
data = pd.Series([1, NA, 3.5, NA, 7])

In [8]:
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

This is equivalent to:

In [9]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

With DataFrame objects, things are a bit more complex. You may want to drop rows
or columns that are all NA or only those containing any NAs. dropna by default drops
any row containing a missing value:

In [10]:
 data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],[NA, NA, NA], [NA, 6.5, 3.]])

In [11]:
cleaned = data.dropna()

In [12]:
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [13]:
cleaned

,0,1,2
0,1.0,6.5,3.0


Passing how='all' will only drop rows that are all NA:

In [14]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


To drop columns in the same way, pass axis=1:

In [15]:
data[4] = NA

In [16]:
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [17]:
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


A related way to filter out DataFrame rows tends to concern time series data. Suppose
you want to keep only rows containing a certain number of observations. You can
indicate this with the thresh argument:

In [18]:
 df = pd.DataFrame(np.random.randn(7, 3))

In [19]:
df.iloc[:4, 1] = NA

In [20]:
df.iloc[:2, 2] = NA
df

,0,1,2
0,1.716190,NaN,NaN
1,0.758215,NaN,NaN
2,1.458738,NaN,-0.222721
3,0.018101,NaN,-1.843687
4,1.125204,1.644142,-0.209151
5,-1.465942,0.080446,-0.432103
6,-1.684639,-1.514129,-2.032758


In [21]:
df.dropna()

,0,1,2
4,1.125204,1.644142,-0.209151
5,-1.465942,0.080446,-0.432103
6,-1.684639,-1.514129,-2.032758


In [22]:
df.dropna(thresh=2)

,0,1,2
2,1.458738,NaN,-0.222721
3,0.018101,NaN,-1.843687
4,1.125204,1.644142,-0.209151
5,-1.465942,0.080446,-0.432103
6,-1.684639,-1.514129,-2.032758


## Filling In Missing Data

Rather than filtering out missing data (and potentially discarding other data along
with it), you may want to fill in the “holes” in any number of ways. For most purposes, the fillna method is the workhorse function to use. Calling fillna with a
constant replaces missing values with that value:

In [23]:
 df.fillna(0)

,0,1,2
0,1.716190,0.000000,0.000000
1,0.758215,0.000000,0.000000
2,1.458738,0.000000,-0.222721
3,0.018101,0.000000,-1.843687
4,1.125204,1.644142,-0.209151
5,-1.465942,0.080446,-0.432103
6,-1.684639,-1.514129,-2.032758


Calling fillna with a dict, you can use a different fill value for each column:

In [24]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,1.716190,0.500000,0.000000
1,0.758215,0.500000,0.000000
2,1.458738,0.500000,-0.222721
3,0.018101,0.500000,-1.843687
4,1.125204,1.644142,-0.209151
5,-1.465942,0.080446,-0.432103
6,-1.684639,-1.514129,-2.032758


fillna returns a new object, but you can modify the existing object in-place:

In [25]:
 _ = df.fillna(0, inplace=True)

In [26]:
df

,0,1,2
0,1.716190,0.000000,0.000000
1,0.758215,0.000000,0.000000
2,1.458738,0.000000,-0.222721
3,0.018101,0.000000,-1.843687
4,1.125204,1.644142,-0.209151
5,-1.465942,0.080446,-0.432103
6,-1.684639,-1.514129,-2.032758


The same interpolation methods available for reindexing can be used with fillna:

In [27]:
 df = pd.DataFrame(np.random.randn(6, 3))

In [28]:
df.iloc[2:, 1] = NA

In [29]:
df.iloc[4:, 2] = NA

In [30]:
df

,0,1,2
0,-0.403981,2.596961,0.373623
1,-0.576694,0.154581,1.233917
2,-0.045495,NaN,0.360516
3,-1.928080,NaN,1.081916
4,1.323195,NaN,NaN
5,1.184422,NaN,NaN


In [31]:
df.fillna(method='ffill')

,0,1,2
0,-0.403981,2.596961,0.373623
1,-0.576694,0.154581,1.233917
2,-0.045495,0.154581,0.360516
3,-1.928080,0.154581,1.081916
4,1.323195,0.154581,1.081916
5,1.184422,0.154581,1.081916


In [32]:
df.fillna(method='ffill', limit=2)

,0,1,2
0,-0.403981,2.596961,0.373623
1,-0.576694,0.154581,1.233917
2,-0.045495,0.154581,0.360516
3,-1.928080,0.154581,1.081916
4,1.323195,NaN,1.081916
5,1.184422,NaN,1.081916


With fillna you can do lots of other things with a little creativity. For example, you
might pass the mean or median value of a Series

In [33]:
data = pd.Series([1., NA, 3.5, NA, 7])

In [34]:
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

Table 7-2. fillna function arguments
--------------------------------------------------
Argument  --> Description
   
   
value --> Scalar value or dict-like object to use to fill missing values
   
   
method --> Interpolation; by default 'ffill' if function called with no other arguments
     
axis --> Axis to fill on; default axis=0
     
inplace --> Modify the calling object without producing a copy
       
limit --> For forward and backward filling, maximum number of consecutive periods to fill

## 7.2 Data Transformation


### Removing Duplicates

In [35]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


The DataFrame method duplicated returns a boolean Series indicating whether each
row is a duplicate (has been observed in a previous row) or not:

In [36]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

Relatedly, drop_duplicates returns a DataFrame where the duplicated array is False:

In [37]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


Both of these methods by default consider all of the columns; alternatively, you can
specify any subset of them to detect duplicates. Suppose we had an additional column
of values and wanted to filter duplicates only based on the 'k1' column:

In [38]:
data['v1'] = range(7)

In [39]:
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [40]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


duplicated and drop_duplicates by default keep the first observed value combination. Passing keep='last' will return the last one:

In [41]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### Transforming Data Using a Function or Mapping


For many datasets, you may wish to perform some transformation based on the values in an array, Series, or column in a DataFrame. Consider the following hypothetical data collected about various kinds of meat:

In [42]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon','Pastrami', 'corned beef', 'Bacon','pastrami', 'honey ham', 'nova lox'],'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})

In [43]:
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


Suppose you wanted to add a column indicating the type of animal that each food
came from. Let’s write down a mapping of each distinct meat type to the kind of
animal:

In [44]:
meat_to_animal = {
 'bacon': 'pig',
 'pulled pork': 'pig',
 'pastrami': 'cow',
 'corned beef': 'cow',
 'honey ham': 'pig',
 'nova lox': 'salmon'
}

The map method on a Series accepts a function or dict-like object containing a mapping, but here we have a small problem in that some of the meats are capitalized and
others are not. Thus, we need to convert each value to lowercase using the str.lower
Series method

In [45]:
lowercased = data['food'].str.lower()

In [46]:
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [47]:
data['animal'] = lowercased.map(meat_to_animal)

In [48]:
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


We could also have passed a function that does all the work:

In [49]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

Using map is a convenient way to perform element-wise transformations and other
data cleaning–related operations.

## Replacing Values

Filling in missing data with the fillna method is a special case of more general value
replacement. As you’ve already seen, map can be used to modify a subset of values in
an object but replace provides a simpler and more flexible way to do so. Let’s consider this Series:

In [50]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])

In [51]:
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

The -999 values might be sentinel values for missing data. To replace these with NA
values that pandas understands, we can use replace, producing a new Series (unless
you pass inplace=True):


In [52]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

If you want to replace multiple values at once, you instead pass a list and then the
substitute value:

In [53]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

To use a different replacement for each value, pass a list of substitutes:

In [54]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

The argument passed can also be a dict:

In [55]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

The data.replace method is distinct from data.str.replace,
which performs string substitution element-wise. We look at these
string methods on Series later in the chapter.

## Renaming Axis Indexes

Like values in a Series, axis labels can be similarly transformed by a function or mapping of some form to produce new, differently labeled objects. You can also modify
the axes in-place without creating a new data structure. Here’s a simple example:

In [56]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),index=['Ohio', 'Colorado', 'New York'],columns=['one', 'two', 'three', 'four'])

In [57]:
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


Like a Series, the axis indexes have a map method:

In [58]:
transform = lambda x: x[:4].upper()

In [59]:
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

You can assign to index, modifying the DataFrame in-place:

In [60]:
data.index = data.index.map(transform)

In [61]:
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


If you want to create a transformed version of a dataset without modifying the original, a useful method is rename:

In [62]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


Notably, rename can be used in conjunction with a dict-like object providing new values for a subset of the axis labels:

In [63]:
data.rename(index={'OHIO': 'INDIANA'},columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


rename saves you from the chore of copying the DataFrame manually and assigning
to its index and columns attributes. Should you wish to modify a dataset in-place,
pass inplace=True:


In [64]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)

In [65]:
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### Discretization and Binning

Continuous data is often discretized or otherwise separated into “bins” for analysis.
Suppose you have data about a group of people in a study, and you want to group
them into discrete age buckets:

In [66]:
 ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

Let’s divide these into bins of 18 to 25, 26 to 35, 36 to 60, and finally 61 and older. To
do so, you have to use cut, a function in pandas:

In [67]:
bins = [18, 25, 35, 60, 100]

In [68]:
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

The object pandas returns is a special Categorical object. The output you see
describes the bins computed by pandas.cut. You can treat it like an array of strings
indicating the bin name; internally it contains a categories array specifying the distinct category names along with a labeling for the ages data in the codes attribute:

In [69]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [70]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

In [71]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

Note that pd.value_counts(cats) are the bin counts for the result of pandas.cut.   
     
Consistent with mathematical notation for intervals, a parenthesis means that the side
is open, while the square bracket means it is closed (inclusive). You can change which
side is closed by passing right=False:

In [72]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

You can also pass your own bin names by passing a list or array to the labels option:

In [73]:
 group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']

In [74]:
pd.cut(ages, bins, labels=group_names)


['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

If you pass an integer number of bins to cut instead of explicit bin edges, it will compute equal-length bins based on the minimum and maximum values in the data.
Consider the case of some uniformly distributed data chopped into fourths:

In [75]:
data = np.random.rand(20)

In [76]:
data

array([0.73537446, 0.45852963, 0.242461  , 0.6798634 , 0.21123872,
       0.51646697, 0.01183484, 0.23061445, 0.01605296, 0.59781633,
       0.61254167, 0.15803606, 0.84163006, 0.48022942, 0.37551369,
       0.43149055, 0.73851027, 0.92030394, 0.38014579, 0.06505485])

In [77]:
pd.cut(data, 4, precision=2)


[(0.69, 0.92], (0.24, 0.47], (0.24, 0.47], (0.47, 0.69], (0.011, 0.24], ..., (0.24, 0.47], (0.69, 0.92], (0.69, 0.92], (0.24, 0.47], (0.011, 0.24]]
Length: 20
Categories (4, interval[float64]): [(0.011, 0.24] < (0.24, 0.47] < (0.47, 0.69] < (0.69, 0.92]]

The precision=2 option limits the decimal precision to two digits.
    
     
A closely related function, qcut, bins the data based on sample quantiles. Depending
on the distribution of the data, using cut will not usually result in each bin having the
same number of data points. Since qcut uses sample quantiles instead, by definition
you will obtain roughly equal-size bins:

In [78]:
data = np.random.randn(1000) # Normally distributed

In [79]:
data

array([-1.14487773e+00,  7.88131466e-01, -8.60069474e-01, -9.51275040e-01,
        9.89472165e-01,  8.26985859e-01, -1.42521374e-01, -6.22141446e-01,
        1.33237531e+00,  1.00816123e+00,  1.43794115e-01,  1.07646329e+00,
        6.81086622e-06, -8.58980451e-01,  3.16332186e-01, -9.34486234e-01,
       -2.78908437e-01, -2.48190208e-01, -7.95160838e-01, -3.40100686e-01,
        8.86265755e-01,  1.39985044e+00, -5.46725605e-01, -1.22923968e+00,
        8.70843388e-01, -1.38884475e+00, -4.43779483e-02, -9.14219558e-01,
       -4.93813310e-01, -9.12873467e-02,  3.94067834e-01, -8.72498594e-01,
       -9.25801681e-01, -7.79585447e-01, -3.52987508e-01,  1.88336451e+00,
       -6.89650465e-02, -4.34866855e-01,  1.10144202e+00, -8.45829224e-01,
       -7.85241005e-01,  1.40796477e-01,  5.89757591e-01,  1.73502550e+00,
       -3.88629648e-01, -5.76089447e-01,  6.14487853e-01,  2.14663187e+00,
       -3.04455616e-01, -2.08384938e+00,  7.84835849e-01,  4.94567626e-01,
       -2.70977991e-01, -

In [80]:
cats = pd.qcut(data, 4) # Cut into quartiles

In [81]:
cats

[(-3.895, -0.74], (0.63, 2.635], (-3.895, -0.74], (-3.895, -0.74], (0.63, 2.635], ..., (-0.009, 0.63], (-0.74, -0.009], (0.63, 2.635], (0.63, 2.635], (-3.895, -0.74]]
Length: 1000
Categories (4, interval[float64]): [(-3.895, -0.74] < (-0.74, -0.009] < (-0.009, 0.63] < (0.63, 2.635]]

In [84]:
pd.value_counts(cats)

(0.63, 2.635]      250
(-0.009, 0.63]     250
(-0.74, -0.009]    250
(-3.895, -0.74]    250
dtype: int64

Similar to cut you can pass your own quantiles (numbers between 0 and 1, inclusive):

In [85]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

[(-1.308, -0.009], (-0.009, 1.192], (-1.308, -0.009], (-1.308, -0.009], (-0.009, 1.192], ..., (-0.009, 1.192], (-1.308, -0.009], (-0.009, 1.192], (-0.009, 1.192], (-3.895, -1.308]]
Length: 1000
Categories (4, interval[float64]): [(-3.895, -1.308] < (-1.308, -0.009] < (-0.009, 1.192] < (1.192, 2.635]]

### Detecting and Filtering Outliers

Filtering or transforming outliers is largely a matter of applying array operations.
Consider a DataFrame with some normally distributed data:

In [86]:
data = pd.DataFrame(np.random.randn(1000, 4))

In [87]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.022110,-0.043906,-0.037331,-0.054070
std,1.014601,1.007231,1.015098,0.954552
min,-3.182219,-3.420015,-3.368210,-2.689002
25%,-0.696666,-0.714993,-0.727664,-0.699930
50%,-0.036531,-0.052193,-0.068682,-0.045962
75%,0.655823,0.636319,0.658800,0.585116
max,3.686829,3.830218,3.063229,2.890968


Suppose you wanted to find values in one of the columns exceeding 3 in absolute
value:

In [88]:
col = data[2]

In [89]:
col

0     -1.009705
1      1.020642
2      0.003759
3      0.947368
4      2.037514
         ...   
995   -0.065644
996   -0.119238
997    2.334061
998   -2.849141
999    0.052484
Name: 2, Length: 1000, dtype: float64

In [90]:
col[np.abs(col) > 3]

479   -3.368210
694    3.063229
Name: 2, dtype: float64

To select all rows having a value exceeding 3 or –3, you can use the any method on a
boolean DataFrame:

In [91]:
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
52,1.374740,3.385276,-1.629590,1.183052
104,-3.182219,0.191703,0.838626,-0.155851
280,-0.682679,3.830218,1.963083,-1.602334
293,0.440736,-3.055962,-0.747369,-0.386925
321,-0.930775,-3.306304,1.968698,0.281366
325,3.686829,0.940408,0.183505,-0.853655
371,0.817982,3.102559,0.601612,0.036518
398,-3.027455,2.411899,-0.397356,0.705882
468,3.136631,0.837253,-0.433790,-2.087663
479,0.767217,-0.410861,-3.368210,0.982346


Values can be set based on these criteria. Here is code to cap values outside the interval –3 to 3:

In [92]:
data[np.abs(data) > 3] = np.sign(data) * 3

In [93]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.022818,-0.044442,-0.037026,-0.054070
std,1.010820,1.000364,1.013763,0.954552
min,-3.000000,-3.000000,-3.000000,-2.689002
25%,-0.696666,-0.714993,-0.727664,-0.699930
50%,-0.036531,-0.052193,-0.068682,-0.045962
75%,0.655823,0.636319,0.658800,0.585116
max,3.000000,3.000000,3.000000,2.890968


The statement np.sign(data) produces 1 and –1 values based on whether the values
in data are positive or negative:

In [94]:
np.sign(data).head()

,0,1,2,3
0,1.0,1.0,-1.0,1.0
1,-1.0,-1.0,1.0,-1.0
2,-1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0
4,-1.0,1.0,1.0,1.0


### Permutation and Random Sampling

Permuting (randomly reordering) a Series or the rows in a DataFrame is easy to do
using the numpy.random.permutation function. Calling permutation with the length
of the axis you want to permute produces an array of integers indicating the new
ordering:

In [95]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [96]:
sampler = np.random.permutation(5)
sampler

array([3, 2, 1, 4, 0])

That array can then be used in iloc-based indexing or the equivalent take function:

In [97]:
df.take(sampler)

,0,1,2,3
3,12,13,14,15
2,8,9,10,11
1,4,5,6,7
4,16,17,18,19
0,0,1,2,3


To select a random subset without replacement, you can use the sample method on
Series and DataFrame:

In [98]:
df.sample(n=3)

,0,1,2,3
0,0,1,2,3
2,8,9,10,11
1,4,5,6,7


To generate a sample with replacement (to allow repeat choices), pass replace=True
to sample:

In [99]:
choices = pd.Series([5, 7, -1, 6, 4])

In [100]:
draws = choices.sample(n=10, replace=True)
draws

4    4
1    7
2   -1
4    4
1    7
2   -1
1    7
3    6
2   -1
3    6
dtype: int64

### Computing Indicator/Dummy Variables

Another type of transformation for statistical modeling or machine learning applications is converting a categorical variable into a “dummy” or “indicator” matrix. If a
column in a DataFrame has k distinct values, you would derive a matrix or DataFrame with k columns containing all 1s and 0s. pandas has a get_dummies function
for doing this, though devising one yourself is not difficult. Let’s return to an earlier
example DataFrame:

In [101]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],'data1': range(6)})


In [102]:
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [103]:
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In some cases, you may want to add a prefix to the columns in the indicator DataFrame, which can then be merged with the other data. get_dummies has a prefix argument for doing this:


In [104]:
dummies = pd.get_dummies(df['key'], prefix='key')

In [105]:
df_with_dummy = df[['data1']].join(dummies)

In [106]:
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


If a row in a DataFrame belongs to multiple categories, things are a bit more complicated. Let’s look at the MovieLens 1M dataset, which is investigated in more detail in
Chapter 14:

In [107]:
mnames = ['movie_id', 'title', 'genres']

In [108]:
movies = pd.read_table('movies.dat', sep='::',header=None, names=mnames)

C:\Users\ankit19.gupta\OneDrive - Reliance Corporate IT Park Limited\Desktop\Practice_Code\Python_Practice\Python_For_Data_Analysis\myenv\lib\site-packages\pandas\io\parsers.py:767: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return read_csv(**locals())


In [109]:
movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


Adding indicator variables for each genre requires a little bit of wrangling. First, we
extract the list of unique genres in the dataset:

In [110]:
all_genres = []

In [111]:
for x in movies.genres:
    all_genres.extend(x.split('|'))

In [112]:
genres = pd.unique(all_genres)

In [113]:
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

One way to construct the indicator DataFrame is to start with a DataFrame of all
zeros:

In [114]:
zero_matrix = np.zeros((len(movies), len(genres)))

In [115]:
zero_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [116]:
dummies = pd.DataFrame(zero_matrix, columns=genres)
dummies

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3879,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3880,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3881,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now, iterate through each movie and set entries in each row of dummies to 1. To do
this, we use the dummies.columns to compute the column indices for each genre:

In [117]:
gen = movies.genres[0]

In [118]:
gen.split('|')

['Animation', "Children's", 'Comedy']

In [119]:
dummies.columns.get_indexer(gen.split('|'))

array([0, 1, 2], dtype=int64)

Then, we can use .iloc to set values based on these indices:

In [120]:
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

Then, as before, you can combine this with movies:

In [121]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))

In [122]:
movies_windic.iloc[0]

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Animation                                1
Genre_Children's                               1
Genre_Comedy                                   1
Genre_Adventure                                0
Genre_Fantasy                                  0
Genre_Romance                                  0
Genre_Drama                                    0
Genre_Action                                   0
Genre_Crime                                    0
Genre_Thriller                                 0
Genre_Horror                                   0
Genre_Sci-Fi                                   0
Genre_Documentary                              0
Genre_War                                      0
Genre_Musical                                  0
Genre_Mystery                                  0
Genre_Film-Noir                                0
Genre_Western       

For much larger data, this method of constructing indicator vari‐
ables with multiple membership is not especially speedy. It would
be better to write a lower-level function that writes directly to a
NumPy array, and then wrap the result in a DataFrame.

A useful recipe for statistical applications is to combine get_dummies with a discreti‐
zation function like cut:

In [123]:
np.random.seed(12345)

In [124]:
values = np.random.rand(10)

In [125]:
values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [126]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]

In [127]:
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


## 7.3 String Manipulation

### String Object Methods

In [128]:
val = 'a,b, guido'

In [129]:
val.split(',')

['a', 'b', ' guido']

split is often combined with strip to trim whitespace (including line breaks):

In [130]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

These substrings could be concatenated together with a two-colon delimiter using
addition:

In [131]:
first, second, third = pieces

In [132]:
first + '::' + second + '::' + third

'a::b::guido'

But this isn’t a practical generic method. A faster and more Pythonic way is to pass a
list or tuple to the join method on the string '::':

In [133]:
'::'.join(pieces)

'a::b::guido'

Other methods are concerned with locating substrings. Using Python’s in keyword is
the best way to detect a substring, though index and find can also be used:

In [134]:
'guido' in val

True

In [135]:
val.index(',')

1

In [137]:
val.find(':')

-1

Note the difference between find and index is that index raises an exception if the
string isn’t found (versus returning –1):

In [138]:
val.index(':')

ValueError: substring not found

Relatedly, count returns the number of occurrences of a particular substring:

In [139]:
val.count(',')

2

replace will substitute occurrences of one pattern for another. It is commonly used
to delete patterns, too, by passing an empty string:

In [141]:
val.replace(',', '::')

'a::b:: guido'

In [142]:
val.replace(',', '')


'ab guido'

Table 7-3. Python built-in string methods
------------------------------------------------------------
Argument --> Description
   
    
count --> Return the number of non-overlapping occurrences of substring in the string. 
     
endswith --> Returns True if string ends with sux.
   
    
startswith --> Returns True if string starts with prefix.
   
    
join --> Use string as delimiter for concatenating a sequence of other strings.
  
    
index --> Return position of first character in substring if found in the string; raises ValueError if not found.
  
    
find --> Return position of first character of rst occurrence of substring in the string; like index, but returns –1
if not found.
   
    
rfind --> Return position of first character of last occurrence of substring in the string; returns –1 if not found.
  
    
replace --> Replace occurrences of string with another string.
   
    
strip,rstrip,lstrip --> Trim whitespace, including newlines; equivalent to x.strip() (and rstrip, lstrip, respectively)
for each element.
   
    
split --> Break string into list of substrings using passed delimiter.
  
    
lower --> Convert alphabet characters to lowercase.
   
    
upper --> Convert alphabet characters to uppercase.
   
    
casefold --> Convert characters to lowercase, and convert any region-specific variable character combinations to a
common comparable form.
  
    
ljust,rjust --> Left justify or right justify, respectively; pad opposite side of string with spaces (or some other fill
character) to return a string with a minimum width.

### Regular Expressions

Regular expressions provide a flexible way to search or match (often more complex)
string patterns in text. A single expression, commonly called a regex, is a string
formed according to the regular expression language. Python’s built-in re module is
responsible for applying regular expressions to strings; I’ll give a number of examples
of its use here.

The re module functions fall into three categories: pattern matching, substitution,
and splitting. Naturally these are all related; a regex describes a pattern to locate in the
text, which can then be used for many purposes. Let’s look at a simple example:

suppose we wanted to split a string with a variable number of whitespace characters
(tabs, spaces, and newlines). The regex describing one or more whitespace characters
is \s+:

In [144]:
import re

In [145]:
text = "foo bar\t baz \tqux"

In [146]:
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

When you call re.split('\s+', text), the regular expression is first compiled, and
then its split method is called on the passed text. You can compile the regex yourself
with re.compile, forming a reusable regex object:

In [147]:
regex = re.compile('\s+')

In [148]:
regex.split(text)

['foo', 'bar', 'baz', 'qux']

If, instead, you wanted to get a list of all patterns matching the regex, you can use the
findall method:


In [149]:
regex.findall(text)

[' ', '\t ', ' \t']

To avoid unwanted escaping with \ in a regular expression, use raw
string literals like r'C:\x' instead of the equivalent 'C:\\x'.

Creating a regex object with re.compile is highly recommended if you intend to
apply the same expression to many strings; doing so will save CPU cycles.
   
     
match and search are closely related to findall. While findall returns all matches
in a string, search returns only the first match. More rigidly, match only matches at
the beginning of the string. As a less trivial example, let’s consider a block of text and
a regular expression capable of identifying most email addresses:


In [150]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
     
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'
      
# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

In [152]:
regex

re.compile(r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}', re.IGNORECASE|re.UNICODE)

Using findall on the text produces a list of the email addresses:


In [153]:
regex.findall(text)


['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

search returns a special match object for the first email address in the text. For the
preceding regex, the match object can only tell us the start and end position of the
pattern in the string:

In [154]:
m = regex.search(text)

In [155]:
m

<_sre.SRE_Match object; span=(5, 20), match='dave@google.com'>

In [156]:
text[m.start():m.end()]

'dave@google.com'

regex.match returns None, as it only will match if the pattern occurs at the start of the
string:

In [157]:
print(regex.match(text))

None


Relatedly, sub will return a new string with occurrences of the pattern replaced by the
a new string:

In [158]:
 print(regex.sub('REDACTED', text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



Suppose you wanted to find email addresses and simultaneously segment each
address into its three components: username, domain name, and domain suffix. To
do this, put parentheses around the parts of the pattern to segment:

In [159]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'

In [160]:
regex = re.compile(pattern, flags=re.IGNORECASE)

A match object produced by this modified regex returns a tuple of the pattern com‐
ponents with its groups method:

In [161]:
m = regex.match('wesm@bright.net')
m.groups()

('wesm', 'bright', 'net')

findall returns a list of tuples when the pattern has groups:

In [162]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

sub also has access to groups in each match using special symbols like \1 and \2. The
symbol \1 corresponds to the first matched group, \2 corresponds to the second, and
so forth:

In [163]:
 print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



Table 7-4. Regular expression methods
-------------------------------------   
   
Argument ---> Description   
   
findall ---> Return all non-overlapping matching patterns in a string as a list
   
     
finditer ---> Like findall, but returns an iterator
   
     
match ---> Match pattern at start of string and optionally segment pattern components into groups; if the pattern
matches, returns a match object, and otherwise None
  
    
search ---> Scan string for match to pattern; returning a match object if so; unlike match, the match can be anywhere in
the string as opposed to only at the beginning
    
      
split ---> Break string into pieces at each occurrence of pattern
  
    
sub, subn ---> Replace all (sub) or first n occurrences (subn) of pattern in string with replacement expression; use symbols
\1, \2, ... to refer to match group elements in the replacement string

### Vectorized String Functions in pandas

Cleaning up a messy dataset for analysis often requires a lot of string munging and
regularization. To complicate matters, a column containing strings will sometimes
have missing data:

In [164]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com','Rob': 'rob@gmail.com', 'Wes': np.nan}

In [165]:
data = pd.Series(data)
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [166]:
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

You can apply string and regular expression methods can be applied (passing a
lambda or other function) to each value using data.map, but it will fail on the NA
(null) values. To cope with this, Series has array-oriented methods for string operations that skip NA values. These are accessed through Series’s str attribute; for example, we could check whether each email address has 'gmail' in it with str.contains:

In [167]:
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

Regular expressions can be used, too, along with any re options like IGNORECASE:

In [168]:
pattern

'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'

In [169]:
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

There are a couple of ways to do vectorized element retrieval. Either use str.get or
index into the str attribute:

In [170]:
matches = data.str.match(pattern, flags=re.IGNORECASE)


In [171]:
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

To access elements in the embedded lists, we can pass an index to either of these
functions:

In [172]:
matches.str.get(1)

AttributeError: Can only use .str accessor with string values!

In [173]:
matches.str[0]


AttributeError: Can only use .str accessor with string values!

You can similarly slice strings using this syntax:


In [174]:
data.str[:5]


Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

Table 7-5. Partial listing of vectorized string methods
---------------------------------------------------------------------------
     
Method --> Description
   
    
cat --> Concatenate strings element-wise with optional delimiter
    
contains --> Return boolean array if each string contains pattern/regex
   
    
count --> Count occurrences of pattern
   
    
extract --> Use a regular expression with groups to extract one or more strings from a Series of strings; the result
will be a DataFrame with one column per group
  
    
endswith --> Equivalent to x.endswith(pattern) for each element
  
    
startswith --> Equivalent to x.startswith(pattern) for each element
  
     
findall --> Compute list of all occurrences of pattern/regex for each string
  
    
get --> Index into each element (retrieve i-th element)
  
    
isalnum --> Equivalent to built-in str.alnum
   
    
isalpha --> Equivalent to built-in str.isalpha
   
     
isdecimal --> Equivalent to built-in str.isdecimal
   
    
isdigit --> Equivalent to built-in str.isdigit
   
     
islower --> Equivalent to built-in str.islower
   
     
isnumeric --> Equivalent to built-in str.isnumeric
   
     
isupper --> Equivalent to built-in str.isupper
   
     
join --> Join strings in each element of the Series with passed separator

   
    
len --> Compute length of each string
  
    
lower, upper --> Convert cases; equivalent to x.lower() or x.upper() for each element
  
   
match --> Use re.match with the passed regular expression on each element, returning matched groups as list
   
    
pad --> Add whitespace to left, right, or both sides of strings
   
    
center --> Equivalent to pad(side='both')
   
    
repeat --> Duplicate values (e.g., s.str.repeat(3) is equivalent to x * 3 for each string)
   
     
replace --> Replace occurrences of pattern/regex with some other string
  
    
slice --> Slice each string in the Series
   
    
split --> Split strings on delimiter or regular expression
   
    
strip --> Trim whitespace from both sides, including newlines
   
    
rstrip --> Trim whitespace on right side
  
    
lstrip --> Trim whitespace on left side
    